<a href="https://colab.research.google.com/github/rcolomina/52_deck_card_recognition/blob/master/convolution_algorithms_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performance Analysis of 1D Convolution

The aim of this notebook is to analyse the performance of the different convolution algorithms in the context of COBRA processor.

A comparison will be done considering parallelization with/without GPU processing.





## Definition of Convolution

Convolution is a binary mathematical operations that mixture two functions resulting in another function $(f * g)$ in the same space of functions.

In the general case convolution is defined as follows:

$(f * g)(t) := \int_{-∞}^{+∞} f(τ)g(t-\tau) d\tau. $

Its discrete version using arrays:

$(x * y)[n] := Σ_{j=-∞}^{+∞} x[n]y[n-j] $

## Interpretation

One way to see convolution is as an operaton between signals (i.e. images, audio, radio). The 1st function is seen as the "signal", and the 2nd one is known as the kernel. The former is a fix signal, and the latter is seen as a sliding window that is overlapping the signal domain, depending on the values in which the convolution is calculated (shifting).

## Convolution of Probability Distribution

One use case of Convolution operations, is to calculate the probability distribution of the sum $X+Y$ of two random variables $X$ and $Y$ independently distributed.

When working with discrite probability distributions, the formula of above can be expressed in terms of probabilities as follows:

$P(Z = k) = Σ_{k=-∞}^{+∞} P(X = k) P(Y= z - k)$

COBRA processor uses the aforementioned comlution, returning a $N+M-1$ dimensonal array for inputs of dimensions $N$ and $M$. This is a.k.a. full convolution.


## Speeding Up Convolution with FFT

The discrete Fourier transform (DFT) maps a complex-valued vector $x_k$ (time domain) into its frequency representation given by:

$X_k = Σ_{n=0}^{N-1} x_n e^{-2\pi i k_n / N}$

This transformation allows to calculate to improve the multiplication speed on frequency domain.

Depending on the value of N, different algorithms are deployed for the best performance

## Convolution Padding

Due to the domain of the functions involved on the convoluiton are defined over all the real numbers, is important to pad with zeros the input functions (or arrays) in order to be able to calculate a full convolution period.

For a input signal of size The dimension of the output will be $D_s + D_k - 1$, where $D_s$ and $D_k$ are the dimensions of the signal and the kernel respectively.

## Performance of the Convolution Algorithms

The performance of the discrete convolution algorithms will be done depending the input size of the arrays. In the case of convoluiton, we have two arrys that may or not have the same input size.


In this notebook we will the following ones:

* Numpy Implementation
* Numba Implementation
* Cross-Product customized using Numba
* FFT Convolution from Scipy
* FFT Convoluiton customized using Numba



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import time
from numba import njit
import numpy as np
import numba as nb
from scipy import signal

!export NUMBA_FORCE_CUDA_CC=1


In [ ]:

import numpy as np
from scipy import signal
rng = np.random.default_rng()
sig = rng.standard_normal(1000)
autocorr = signal.fftconvolve(sig, sig[::-1], mode='full')
autocorr
type(rng)

In [ ]:
sig = rng.standard_normal(5)
sig[::-1]
print(sig,sig[::-1])

# Analysis of Convolution Performance for Numba vs Numpy

In [ ]:
def npFftConvolve(a,b):
    return signal.fftconvolve(a, b[::-1], mode='full')

def npConvolve(a, b):
    return np.convolve(a, b)

@nb.njit('float64[:](float64[:], float64[:])')
def nbConvolveUncont(a, b):
    return np.convolve(a, b)

@nb.njit('float64[::1](float64[::1], float64[::1])')
def nbConvolveCont(a, b):
    return np.convolve(a, b)

n = 10

a = np.random.random(n)
b = np.random.random(n)

c = npFftConvolve(a, b)
print("Signal", a)
print("Kernel", b)
print("Convolution", c)
print("dim. Signal =", len(a), "dim. Kernel =", len(b), "din. convo =", len(c))

assert abs(npFftConvolve(a,b).all() == npConvolve(a,b).all())
assert abs(npConvolve(a,b).all() == npConvolve(a,b).all())
assert abs(nbConvolveCont(a,b).all() == npConvolve(a,b).all())

In [ ]:
n = 10 ** 3

a = np.random.random(n)
b = np.random.random(n)

%timeit -n 50 npFftConvolve(a, b)
%timeit -n 50 npConvolve(a, b)
%timeit -n 50 nbConvolveUncont(a, b)
%timeit -n 50 nbConvolveCont(a, b)

# Without GPU
#
#251 µs ± 73.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
#257 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
#2.3 ms ± 56 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
#2.11 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [ ]:
@njit
def custom_convolution_1d(signal, kernel):
  n_sig = signal.size
  n_ker = kernel.size
  n_conv = n_sig - n_ker + 1

  rev_kernel = kernel[::-1].copy()
  result = np.zeros(n_conv)
  for i_conv in range(n_conv):
    result[i_conv] = np.dot(signal[i_conv:i_conv+n_ker], rev_kernel)

  return result

n = 5
a = np.random.random(n)
b = np.random.random(n)

signal = a
kernel = b

signal = np.pad(signal,len(kernel)-1)

c = custom_convolution_1d(signal, kernel)

x = np.convolve(a, b, mode='full')

print(c,x)

print("custom convolution")
print(custom_convolution_1d(signal, kernel))
print("numpt convolution")
print(np.convolve(a, b, mode='full'))

assert custom_convolution_1d(signal, kernel).all() == np.convolve(a, b, mode='full').all()

In [ ]:
n = 10 ** 3

a = np.random.random(n)
b = np.random.random(n)

%timeit -n 10 custom_convolution_1d(a, b)

## Analysis of Cupy Convolution

In [ ]:
!nvcc --version

In [ ]:

!pip install cupy-cuda11x

#!export NUMBA_FORCE_CUDA_CC=1

In [ ]:
import cupy

n = 10 ** 1

a = np.random.random(n)
b = np.random.random(n)

signal = a
kernel = b

signal = np.pad(signal,len(kernel)-1)
custom_results = custom_convolution_1d(signal, kernel)

a1 = cupy.array(a)
b1 = cupy.array(b)

assert custom_results.all() == cupy.convolve(a1, b1).all()
print("custom results",custom_results)
print("cupy results",cupy.convolve(a1, b1))

In [ ]:
n = 10 ** 2

a = np.random.random(n)
b = np.random.random(n)

signal = a
kernel = b

signal = np.pad(signal,len(kernel)-1)
c = custom_convolution_1d(signal, kernel)

print("Custom algo")
%timeit -n 10 custom_convolution_1d(signal, kernel)

a1 = cupy.array(a)
b1 = cupy.array(b)
print("Cupy algo")
%timeit -n 10 cupy.convolve(a1, b1)

In [ ]:
n = 2 * 10 ** 3

a = np.random.random(n)
b = np.random.random(n)

signal = a
kernel = b

signal = np.pad(signal,len(kernel)-1)
c = custom_convolution_1d(signal, kernel)

print("Custom Algo")
%timeit -n 10 custom_convolution_1d(signal, kernel)

a1 = cupy.array(a)
b1 = cupy.array(b)
print("Cupy Algo")
%timeit -n 10 cupy.convolve(a1, b1)

## Analysis on Correctness of FFT+Convolution vs. Naked Convolution


In [ ]:
n = 10 ** 1

a = np.random.random(n)
b = np.random.random(n)

print("Input Arrays of dimension ",n)
print(a)
print(b)

disc_pdf_1c = a
disc_pdf_2c = b

disc_pdf_1c_ft = np.fft.fft(disc_pdf_1c)
disc_pdf_2c_ft = np.fft.fft(disc_pdf_2c)

disc_pdf_pdbi_ft = disc_pdf_1c_ft * disc_pdf_2c_ft

disc_pdf_pdbi = np.real(np.fft.ifft(disc_pdf_pdbi_ft))

print("FFT+Convolve")
print(disc_pdf_pdbi)

print("Numpy Convolution")
print(np.convolve(a,b))

a1 = cupy.array(a)
b1 = cupy.array(b)

print("Cupy Convolution")
cupy.convolve(a1, b1)


## 1D convolution with fft





## FFT Convolution Based

In [ ]:
import numpy as np
from numba import njit
import numba

In [ ]:
a = [1,2,3,4,5,6,7,8]
b = [4,5,6,7,8,9,10,11]

import scipy
scipy.signal.fftconvolve(a,b)

In [ ]:
import scipy

r = scipy.signal.fftconvolve(a,b)

print(r)

assert r.all() == results_fft_convolve.all()

r1 = np.convolve(a,b,mode='full')

print(len(a),len(b))
print(r1)

assert r1.all() == results_fft_convolve.all()

print(len(r))

In [ ]:
## TEST STANDARD CONV.. VS Custom fftconcolve
np.random.seed(42)

for n in range(10,1000,10):

  m = 2 * n # change the size of the domains

  a = np.random.random(n)
  b = np.random.random(m)

  r1 = np.convolve(a,b,mode='full')

  a2, b2 = get_padded(a,b)
  assert len(a2) == len(b2)

  r2 = fullNumpyFftConvolve(a2,b2)

  assert r1.all() == r2.all()

In [ ]:
def fullNumpyFftConvolve(f,g):

  ## apply fft to move to frequency domain (Fourier Space in Actuarial terms)
  f1 = np.fft.fft(f)
  f2 = np.fft.fft(g)

  ## complex numbers multiplication
  m = f1 * f2

  ## return to time domain and get the real part
  ifft = np.fft.ifft(m)
  return np.real(ifft)


def get_padded(a,b):

  N = len(a)
  M = len(b)

  a_pad = np.zeros(M-1)
  b_pad = np.zeros(N-1)

  a2 = np.concatenate((a,a_pad))
  b2 = np.concatenate((b,b_pad))
  return a2,b2


a1 = np.array(a)
b1 = np.array(b)

print("fftconvolve no padding")
a2, b2 = get_padded(a1,b1)

r1 = fullNumpyFftConvolve(a2,b2)

assert len(a2)==len(b2)

print(a2, b2)
print("fftconvolve padded")

results_fft_convolve = fullNumpyFftConvolve(a2,b2)
#print("output dimension =",len(restuls_fft_convolve))
#print(restuls_fft_convolve)

In [ ]:
np.random.random(10)

## Speedtest comparing Tensorflow, PyTorch, CuPy and Numpy

Comparing FFT Convolution over each engine




In [ ]:
%matplotlib inline
import tensorflow as tf
import torch
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
# Print numpy see whether mkl/blas is available
np.show_config()

# check the gpu and cuda version
!nvidia-smi

## TENSORFLOW FFT CONVOLUTION

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()
print('TensorFlow: {}'.format(tf.__version__))

def tfFftConvolve(a,b):
  # direct fft
  af = tf.signal.fft(a)
  bf = tf.signal.fft(b)

  # multiply complex numbers
  m = af * bf

  # inverse fft to get back time domain
  ifft = tf.signal.ifft(m)

  # return real part
  return np.real(ifft)


## Testing Tensorflow FFT convolution

In [ ]:
n = 10 ** 1

a = np.random.random(n)
b = np.random.random(n)

print(a,b)
#print(a.size)

r = tfFftConvolve(a,b)
print(len(r),r)

## TORCH FFT CONVOLUTION

In [ ]:
import torch
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## TODO: implement the pytorch version as well
def pyTorchFftConvolveCUDA(a,b):

  a = torch.from_numpy(a).float().to(device)
  b = torch.from_numpy(b).float().to(device)

  # direct fft
  af = torch.fft.fft(a)
  bf = torch.fft.fft(b)

  # multiply complex numbers
  m = af * bf

  # inverse fft to get back time domain
  ifft = torch.fft.ifft(m)

  # r eturn real part
  return np.real(ifft)

## Testing Torch FFT convolution

In [ ]:
n = 10 ** 2

a = np.random.random(n)
b = np.random.random(n)

#print(a,b)
r1 = pyTorchFftConvolveCUDA(a,b)
r2 = tfFftConvolve(a,b)

assert r1.all() == r2.all()

## CUPY FFT CONVOLUTON

In [ ]:
import cupy as cp

cp.fft.config.use_multi_gpus = True

In [ ]:


def cupyFftConvolution(a,b):

  a = cp.array(a)
  b = cp.array(b)

  af = cp.fft.fft(a)
  bf = cp.fft.fft(b)

  m = af * bf

  ifft = cp.fft.ifft(m)

  return np.real(ifft)

In [ ]:
n =10
a = np.random.random(n)
x = cp.array(a)
x

## Testing CuPy FFT convolution


In [ ]:
n = 10 ** 2

a = np.random.random(n)
b = np.random.random(n)

r1 = cupyFftConvolution(a,b)
r2 = tfFftConvolve(a,b)

## check correctnetss
assert r1.all() == r2.all()

## TESTING ALL TOGETHER

In [ ]:
a = 3
n = a * 10 ** 4

import numpy as np

a = np.random.random(n)
b = np.random.random(n)

#%timeit -n 50 npFftConvolve(a, b)
#%timeit -n 50 npFftConvolve(a,b)
%timeit -n 50 tfFftConvolve(a,b)
%timeit -n 50 pyTorchFftConvolveCUDA(a,b)

In [ ]:
powers_of_two = [2**n for n in range(6,15)]
powers_of_two

In [ ]:
size = max(powers_of_two)
aa = np.random.random(size)
bb = np.random.random(size)

sizes = []

npconvolve_times = []
scipy_times = []
npfft_times = []
tensor_times = []
pytorch_times = []
cupy_times = []
numba_times = []

for size in powers_of_two:

  print("Loop size ", size)

  a = aa[0:size]
  b = bb[0:size]

  print("standard")
  t0 = %timeit -n 10 -o npConvolve(a, b)
  print("fft-scipy")
  t1 = %timeit -n 100 -o npFftConvolve(a, b)
  print("fft-numpy")
  t2 = %timeit -n 10 -o fullNumpyFftConvolve(a,b)
  print("fft-tensor")
  t3 = %timeit -n 10 -o tfFftConvolve(a,b)
  print("fft-pytorch")
  t4 = %timeit -n 10 -o pyTorchFftConvolveCUDA(a,b)
  print("fft-cupy")
  t5 = %timeit -n 10 -o cupyFftConvolution(a,b)
  print("fft-numba")
  t6 = %timeit -n 10 -o nbConvolveCont(a, b)

  sizes.append(size)

  npconvolve_times.append(t0.average)
  scipy_times.append(t1)
  npfft_times.append(t2.average)
  tensor_times.append(t3.average)
  pytorch_times.append(t4.average)
  cupy_times.append(t5.average)
  numba_times.append(t6.average)
print("Experiment Finished")

In [ ]:
# importing matplotlib module
import matplotlib.pyplot as plt
plt.style.use('default')

# %matplotlib inline: only draw static
# images in the notebook
%matplotlib inline

import pandas as pd


#assert len(npconvolve_times) == len(npfft_times)
assert len(npfft_times) == len(tensor_times)
assert len(tensor_times) == len(pytorch_times)
assert len(pytorch_times) == len(cupy_times)



df = pd.DataFrame.from_dict({"size": sizes,
                             "npconvolve":[x for x in npconvolve_times],
                             "scipy":[x.average for x in scipy_times],
                             "npfft":[x for x in npfft_times],
                             "tensor":[x for x in tensor_times],
                             "pytorch":[x for x in pytorch_times],
                             "cupy":[x for x in cupy_times],
                             "numba":[x for x in numba_times]})


plt.figure(figsize=(12, 8), dpi=100)

# using plot method to plot open prices.
# in plot method we set the label and color of the curve.
df = df.set_index("size")

df['npconvolve'].plot(label='Numpy Convolve')
df['scipy'].plot(label='Scipy (CPU)')#, xticks=powers_of_two)
df['npfft'].plot(label='Numpy (CPU)')#, xticks=powers_of_two)
df['tensor'].plot(label='Tensor Flow (GPU)') #' , xticks=powers_of_two)
df['pytorch'].plot(label='Pytorch (GPU)') #, xticks=powers_of_two)
df['cupy'].plot(label='Cupy (GPU)') #, xticks=powers_of_two)
df['numba'].plot(label='Numba (GPU)') #, xticks=powers_of_two)



plt.xticks = powers_of_two
#plt.xticks(rotation=90)
plt.title("FFT Convolution Algorithms (FFT + Fourier Space Product + iFFT)")
plt.xlabel('Input Size [ Size(Signal) ~= Size(Kernel) ]')
plt.ylabel('Time (miliseconds)')
plt.yscale('log')
#plt.xscale('log')
plt.legend()
plt.plot(kind="bars")


In [ ]:
df


In [ ]:

df = df.reset_index()
df

In [ ]:
plt.title("FFT Convolution Algorithms (FFT + Fourier Space Product + iFFT)")
plt.xlabel('Input Size [ Size(Signal) ~= Size(Kernel) ]')
#plt.ylabel('Time (miliseconds)')
#plt.yscale('log')
plt.legend()

df[['size','pytorch','npfft','tensor','cupy']].plot(x='size',kind='bar')

In [ ]:
ax = sns.barplot(x="day", y="total_bill", data=tips)

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

df[]